In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
from pytube import YouTube
import pytube
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import math
import cv2
import os
import unicodedata
import string

In [ ]:
# Store data in lists
plot_synopsis = []
names = []
opening_weekend_gross = []
errors = []

pages = [str(i) for i in range(1,140000,250)]
# pages = ['1']
count = 0
#Prepare monitoring of loop
start_time = time()
requests = 0
#for every page
for page in pages:
    #initialls write the files
    plotExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\movie_plots.txt')
    nameExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\movie_names.txt')
    grossExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\movie_gross.txt')
    errorExists = os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\movie_errors.txt')
    requestExists= os.path.isfile('C:\\Users\Brian\Scraper\plot_sums\requests.txt')
    
    if not plotExists:
        moviePlot = open("plot_sums/movie_plots.txt", "w")
        moviePlot.close()
        
    if  not nameExists:
        movieName = open("plot_sums/movie_names.txt", "w")
        movieName.close()
        
    if not grossExists:
        movieGross = open("plot_sums/movie_gross.txt", "w")
        movieGross.close()
        
    if not errorExists:
        movieError = open("plot_sums/movie_errors.txt", "w")
        movieError.close()
    
    if not requestExists:
        movieRequest = open("plot_sums/movie_request.txt", "w")
        movieRequest.close()
        
    #track number of requests
    movieRequest = open("plot_sums/movie_request.txt", "a")
    movieRequest.write('[Count #: {}]'.format(count))
    movieRequest.write("\n")
    movieRequest.close()
    
    #make get request
    if int(page) < 10000:
        response = get("https://www.imdb.com/search/title?title_type=feature&languages=en&count=250&start=" + page, timeout=10)
    #pause the loop
    sleep(randint(8,15))

    #monitor requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    #throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #break the loop if the number of requests is greater than expected
    if requests > 800:
        warn('Number of requests was greater than expected.')
        break

    # parse the content of request
    page_html = BeautifulSoup(response.text, 'html.parser')

    #select all 250 movie containers from a single page
    mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    linkzor = page_html.find("div", {"id": "main"}).div.div.find('div', class_ = 'desc').find("a", class_ = 'lister-page-next next-page')['href']
    next_page = "https://www.imdb.com{}".format(linkzor)
    movieRequest = open("plot_sums/movie_request.txt", "a")
    movieRequest.write('[Request #: {}]'.format(requests + 1))
    movieRequest.write("\n")
    movieRequest.write("[Next Link:{}]".format(linkzor))
    movieRequest.close()
    response = get(next_page, timeout=10)

    # Extract data from indiv. movie containers
    for container in mv_containers:
        count+=1
        if container.find('div', class_ = 'ratings-metascore') is not None:
            #Profile
            profile = container.h3.a['href']
            name = container.h3.a.text

            details = get("https://www.imdb.com/" + profile)
            # parse the content of request
            details_html = BeautifulSoup(details.text, 'html.parser')
            try:
                                
                #opening weekend BO
                bo = details_html.find(text='Opening Weekend USA:').parent.findNext('span').decompose()
                bo = details_html.find(text='Opening Weekend USA:').parent.findNext('span').decompose()
                ow_bo = details_html.find(text='Opening Weekend USA:').parent.parent.text.strip()

                synopsis_link = profile
                sleep(randint(1,3))
                synopsis = get("https://www.imdb.com/" + synopsis_link + "plotsummary")
                synopsis_html = BeautifulSoup(synopsis.text, 'html.parser')

                plot_synopsis_content = synopsis_html.find("ul", {"id": "plot-synopsis-content"}).li.text
                plot_synopsis_content = plot_synopsis_content.strip()
                if plot_synopsis_content[:58] != "It looks like we don't have a Synopsis for this title yet.":
                    opening_weekend_gross.append(ow_bo[20:].replace('$', '').replace(',', '').strip())
                    names.append(name)
                    plot_synopsis.append(plot_synopsis_content)



                # movie_info = pd.DataFrame({'movie': names, 'plot': plot_synopsis, 'gross': opening_weekend_gross })
                # movie_info.to_csv('movie_plots.csv', encoding='utf-8')

                      # write line to output file
                    moviePlot = open("plot_sums/movie_plots.txt", "a")
#                     moviePlot.write('"')
                    moviePlot.write(plot_synopsis_content)
#                     moviePlot.write('"')
                    moviePlot.write(",,,")
                    moviePlot.write("\n")
                    moviePlot.close()

                    movieName = open("plot_sums/movie_names.txt", "a")
                    movieName.write(name)
                    movieName.write(",,,")
                    movieName.close()

                    movieGross = open("plot_sums/movie_gross.txt", "a")
                    gross = ow_bo[20:].replace('$', '').replace(',', '').strip()
                    movieGross.write(gross)
                    movieGross.write(",,,")
                    movieGross.close()
                    


                    print('Request:{}; Movie:{} - {}'.format(requests, count, name))
                    clear_output(wait = True)
            except:
                errors.append(name)
                # movie_errors = pd.DataFrame({'movie': errors})
                # movie_errors.to_csv('movie_plots_errors.csv', encoding='utf-8')
                movieError = open("plot_sums/movie_errors.txt", "a")
#                 movieError.write('"')
                movieError.write(name)
#                 movieError.write('"')
                movieError.write(",,,") 
                movieError.close()
                print('Request:{}; Movie:{} - {} - ERROR'.format(requests, count, name))
                clear_output(wait = True)
moviePlot = open("plot_sums/movie_plots.txt", "a")
moviePlot.write(']')
moviePlot.close()

movieName = open("plot_sums/movie_names.txt", "a")
movieName.write(']')
movieName.close()

movieGross = open("plot_sums/movie_names.txt", "a")
movieGross.write(']')
movieGross.close()

movieError = open("plot_sums/movie_errors.txt", "a")
movieError.write(']')
movieError.close()

Request:5; Movie:1062 - Casino - ERROR


In [11]:
movie_ratings = pd.DataFrame({
                              'profile': profile_pages,
                              'synopsis': plot_synopsis
                             })

print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
profile     15 non-null object
synopsis    15 non-null object
dtypes: object(2)
memory usage: 320.0+ bytes
None


,profile,synopsis
0,/title/tt9024106/,Abby Johnson's story begins with both her pers...
1,/title/tt1488606/,It looks like we don't have a Synopsis for thi...
2,/title/tt8426594/,It looks like we don't have a Synopsis for thi...
3,/title/tt4827558/,It looks like we don't have a Synopsis for thi...
4,/title/tt1727824/,The film opens with preparations being made fo...
5,/title/tt2386490/,The film opens on a dragon trapper ship. Hiccu...
6,/title/tt0167190/,The movie opens with the voice of Professor Br...
7,/title/tt4913966/,It looks like we don't have a Synopsis for thi...
8,/title/tt6966692/,"In 1962 in New York City, Tony ""Tony Lip"" Vall..."
9,/title/tt8085790/,It looks like we don't have a Synopsis for thi...
